In [8]:
%pylab inline

from library import lasagnennet as LN
from library import features as F
from library import scale as S
from library import validate as V
import numpy as np
import theano as TH
import pandas as pa
import lasagne as L

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [9]:
train_feats = pa.read_csv('train_feats.csv')
train_targets = pa.read_csv('train_targets.csv')
val_feats = pa.read_csv('validation_feats.csv')
val_targets = pa.read_csv('validation_targets.csv')

In [10]:
tx = np.asarray(train_feats, dtype=TH.config.floatX)
tyall = np.asarray(train_targets, dtype=TH.config.floatX)
ty = np.asarray(train_targets['LABEL'], dtype=TH.config.floatX).reshape(-1, 1)
vx = np.asarray(val_feats, dtype=TH.config.floatX)
vyall = np.asarray(val_targets, dtype=TH.config.floatX)
vy = np.asarray(val_targets['LABEL'], dtype=TH.config.floatX).reshape(-1, 1)

In [11]:
print vyall.shape

(152282, 4)


In [12]:
idtimes = [0 for i in xrange(3595)]
for i in vyall:
    idtimes[np.int32(i[0])] = max(idtimes[np.int32(i[0])], i[1])

In [13]:
ty = ty.ravel()
vy = vy.ravel()

In [14]:
from sklearn import svm
from sklearn.metrics import roc_curve

In [ ]:
clf = svm.SVC(verbose=True, max_iter=1000, class_weight={0:1, 1:3})
clf.fit(tx, ty)

In [18]:
probs = clf.predict_proba(vx)

AttributeError: predict_proba is not available when probability=False

In [ ]:
yscore = probs[:,1]
fpr, tpr, thresh = roc_curve(vy, yscore)
pyplot.plot(1-fpr, tpr)

In [47]:
thresh = np.percentile(probs[:,1], 99)
pred = np.int32(probs[:,1]>=thresh)
# for i in xrange(len(pred)):
#     id = np.int32(vyall[i,0])
#     if idtimes[id]-vyall[i,1] < 5*60*60:
#         pred[i] = 0
print np.sum(pred)

1523


In [12]:
print np.where(pred==1)

(array([1303, 1305, 1306, 1307, 1308, 1310, 1313, 1317, 1330, 1332, 1351,
       1352, 1359, 1365, 1369, 1380]),)


In [13]:
print np.int32(vyall[19441, 1]), idtimes[986]

1137110 0


In [48]:
output = vyall.swapaxes(0,1)
output[2] = pred
output = output.swapaxes(0,1)

In [51]:
outfile = open('out.csv', 'w')
outfile.write('ID,TIME,LABEL,ICU\n')
for i in output:
    if i[3] == 1:
        outfile.write(','.join([str(int(j)) for j in i])+'\n')
outfile.close()

In [52]:
reload(V)
V.validate('out.csv', './Training_Dataset/id_label_train.csv')

tp :  9 , tn :  831 , fp :  6 , fn :  51
sensitivity :  0.15 , specificity :  0.992831541219
accuracy :  0.936454849498 median prediction time :  135.3575
finalScore :  32.6657706093


(0.15,
 0.992831541218638,
 0.9364548494983278,
 135.35749999999999,
 32.665770609319026)